<h1 style="font-family:Times New Roman;">Расчет матрицы расстояний</h1>

*Nikolay Sitala 2021, nsitala@ukr.net* 
>http://map.project-osrm.org

<h3 style="font-family:Times New Roman;color:gray">Пример файла Excel .xlsx с координатами</h3>

In [52]:
try:
    import piplite
    await piplite.install(['ipywidgets', 'openpyxl', 'pandas', 'requests', 'tqdm', 'ssl'])
except ImportError:
    pass

import doexample
doexample.start("resources/xlsx_coords.test", "coord_example.xlsx")

<h2 style="font-family:Times New Roman;">Загрузите файл .xlsx c координатами</h2>

In [53]:
import ipywidgets as widgets

uploader = widgets.FileUpload(
    accept='.xlsx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
flag_upload = 'xlsx'
uploader = widgets.FileUpload()
display(uploader)

FileUpload(value=(), description='Upload')

<h3 style="font-family: 'Times New Roman'">Предпросмотр .xlsx с координатами</h3>

In [54]:
import io
import openpyxl
from pandas import ExcelFile

uploaded_file = uploader.value[0]
print(uploaded_file.name)
excel_data = ExcelFile(io.BytesIO(uploaded_file.content.tobytes()), engine='openpyxl')
dataframe = excel_data.parse(excel_data.sheet_names[0])
dataframe

coord_example.xlsx


,location,latitude,lontitude
0,"Жерноклевы, ЧЕРКАССКАЯ",49.969062,31.935005
1,"Зарубинцы, ЧЕРКАССКАЯ",49.126683,29.955307
2,"Звенигородка, ЧЕРКАССКАЯ",49.070364,30.968338
3,"Здобуток Жовтня, ЧЕРКАССКАЯ",48.868504,30.749146
4,"Золотоноша, ЧЕРКАССКАЯ",49.669020,32.042598
5,"Ирдынь, ЧЕРКАССКАЯ",49.367147,31.677246
6,"Ирклиев, ЧЕРКАССКАЯ",49.518256,32.338096
7,"Кайтановка, ЧЕРКАССКАЯ",48.891665,31.082583
8,"Канев, ЧЕРКАССКАЯ",49.749587,31.459825


<h3 style="font-family:Times New Roman;color:gray">Пример файла CSV с координатами</h3>

In [ ]:
import io
import pandas as pd
import base64
from IPython.display import FileLink
from ipywidgets import HTML

def create_download_link(df, title = "Скачать пример в CSV file ", filename = "data.csv"):  
    csv = df.to_csv(sep=';', encoding='utf-8-sig', index=False)
    b64 = base64.b64encode(csv.encode('utf-8-sig'))
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

csv_data = pd.read_csv("resources/csv_coords.test", delimiter=';', encoding ='utf-8')
create_download_link(csv_data)

<h2 style="font-family:Times New Roman;color:gray">Загрузите файл формат CSV c координатами</h2>

In [ ]:
import ipywidgets as widgets

uploader = widgets.FileUpload(
    accept='.xlsx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
flag_upload = 'csv'
uploader = widgets.FileUpload()
display(uploader)

<h3 style="font-family:Times New Roman;color:gray">Предпросмотр CSV с координатами</h3>

In [ ]:
import io
import openpyxl
from pandas import ExcelFile
import pandas as pd

uploaded_file = uploader.value[0]
print(uploaded_file.name)
csv_data = pd.read_csv(io.BytesIO(uploaded_file.content.tobytes()), delimiter=';', encoding ='utf-8')
csv_data

[Как преобразовать файлы Excel в формат CSV](https://office-guru.ru/excel/kak-preobrazovat-excel-v-csv-i-eksportirovat-faily-excel-v-format-csv-utf-8-456.html#2>)

<h2 style="font-family: 'Times New Roman';color:blue">Cкрипт создания матрицы расстояний</h2>

In [55]:
trip = widgets.Dropdown(
    options=[('Авто', 1), ('Байк', 2), ('Пешком', 3), ('Euclidian', 4)],
    value=4,
    description='Выберите из списка:',
)
trip

Dropdown(description='Выберите из списка:', index=3, options=(('Авто', 1), ('Байк', 2), ('Пешком', 3), ('Eucli…

<h2 style="font-family: 'Times New Roman';color:blue">Ограничение по количеству расчетов</h2>

In [56]:
calc_limit = widgets.IntText(
    value=0,
    description='Число:',
    disabled=False
)
calc_limit

IntText(value=0, description='Число:')

In [71]:
## http://map.project-osrm.org/
%pip install -q ssl

import os
import time
import json
import pprint
import requests

from openpyxl import Workbook, load_workbook
from openpyxl.worksheet import worksheet
# from tqdm import tqdm

import pandas as pd
from pandas import DataFrame

import wgsdistance


import json
from pyodide import to_js
from IPython.display import JSON
from js import Object, fetch


wb = Workbook()

global trip
global calc_limit

type_driving = "routed-car"

if trip.value == 1:
    type_driving = "routed-car"
elif trip.value == 2:
    type_driving = "routed-bike"
elif trip.value == 3:
    type_driving = "routed-foot"
elif trip.value == 4:
    type_driving = "routed-euclidian"

async def get_distance_osrm(coordinates, coord_list):

    global type_driving
    
    # coordinates = '30.613808,46.471085;29.234447,47.997272'
    # r1 = f'http://router.project-osrm.org/route/v1/{type_driving}/{coordinates}?overview=false'
    r1 = f'https://routing.openstreetmap.de/{type_driving}/route/v1/driving/{coordinates}?overview=false&alternatives=true&steps=false'
    
    i = 8
    
    while i > 0:
        r1 = f'https://routing.openstreetmap.de/{type_driving}/route/v1/driving/{coordinates}?overview=false&alternatives=true&steps=false'
        resp = await fetch(r1, method="GET", headers=Object.fromEntries(to_js({ "Content-Type": "application/json" })),)
        x = resp.headers.get('Cache-Control')
        if x == 'max-age=0':
            break
        else:
            break
            i -= 1
            time.sleep(10)
            
    res = await resp.text()
    res_json = json.loads(res)
    
    return res_json, r1


def import_data_excel():
    
    global excel_data
    
    # wb1 = load_workbook(io.BytesIO(uploader.data[-1]))
    wb1 = load_workbook(io.BytesIO(uploaded_file.content))
    
    sheets_list = wb1.sheetnames
    wsdata = wb1[sheets_list[0]]
    
    use_min_row = 2
    use_min_col = 1
    use_max_col = 3
    use_max_row = wsdata.max_row
    
    row_fields = worksheet.Worksheet.iter_rows(wsdata, min_row=use_min_row,
                                               max_row=use_max_row,
                                               min_col=use_min_col,
                                               max_col=use_max_col)
    for b in row_fields:
        yield b


def get_data_matrix_excel():

    coords_pull = []
    matrix_out = []
    j = 0
    
    for z4 in import_data_excel():
        z4_list = []
        for i, u in enumerate(z4):
            if not u.value is None:
                z4_list.append(u.value)

        z4_list = [u.value for u in z4[:] if not u.value is None]
        if z4_list:
            j += 1
            coords_pull.append([j, z4_list[0], z4_list[1], z4_list[2]])

    first_list = coords_pull
    matrix_out = []

    while first_list:
        s = first_list.pop(0)
        matrix_out.extend([[s, b] for b in [h for h in first_list]])
    return matrix_out


def import_data_csv():
    
    global csv_data
    
    locate = list(csv_data['location'])
    lat = list(csv_data['latitude'])
    lon = list(csv_data['lontitude'])
    
    for b in zip(locate, lat, lon):
        yield(b)


# def get_progress(mtx):
#     for i in tqdm(range(mtx)):
#         yield 0
        
        
def get_data_matrix():

    global flag_upload
    
    if flag_upload == 'csv':
        print('Источник данных .csv')
        j = 0
        coords_pull = []
        matrix_out = []

        for z in import_data_csv():
            j += 1
            coords_pull.append([j, z[0], z[1], z[2]])

        while coords_pull:
            s = coords_pull.pop(0)
            matrix_out.extend([[s, b] for b in [h for h in coords_pull]])
    
    elif flag_upload == 'xlsx':
        print('Источник данных .xlsx')
        matrix_out = get_data_matrix_excel()

    return matrix_out


async def add_data_list(matrix_out):
    
    wb.create_sheet("array")
    wsarray = wb['array']
    
    # br = get_progress(len(matrix_out)*2)
    quit = 0
    
    for b in matrix_out:
        
        if quit >= calc_limit.value and calc_limit.value > 0:
            wsarray.cell(row=b[0][0], column=b[1][0]).value = 0
            # next(br)
            quit += 1
            wsarray.cell(row=b[1][0], column=b[0][0]).value = 0
            # next(br)
            quit += 1
        else:

            r = [str(b[0][3]), str(b[0][2]), str(b[1][3]), str(b[1][2])]
            coordinates = f'{r[0]},{r[1]};{r[2]},{r[3]}'
            coord_list = [float(r[1]), float(r[0]), float(r[3]), float(r[2])]
            coord_one = [coord_list[0], coord_list[1]]
            coord_two = [coord_list[2], coord_list[3]]
            
            if type_driving == "routed-euclidian":
                dist_km, ang = wgsdistance.distpointwgs84(coord_one, coord_two)
                wsarray.cell(row=b[1][0], column=b[0][0]).value = round(dist_km / 1000, 2)
                wsarray.cell(row=b[0][0], column=b[1][0]).value = round(dist_km / 1000, 2)
                # print('Ok', coord_one, coord_two, 'Euclidian')
                
                for m in range(0, 2):
                    # next(br)
                    quit += 1
    
            else:
                res, web_link = await get_distance_osrm(coordinates, coord_list)
                
                if res['code'] == 'Ok':
                    
                    y = res['routes'][0]
                    legs = y['legs'][0]
                    dist_meters = legs['distance']
                    dist_km = round(dist_meters / 1000, 3)

                    wsarray.cell(row=b[0][0], column=b[1][0]).value = dist_km
                    # print(b[0][0], b[0][1], b[1][0], b[1][1], dist_km)
                else:
                    print(res['code'], coordinates, web_link)
                    wsarray.cell(row=b[0][0], column=b[1][0]).value = 0

                # next(br)
                quit += 1

                r = [str(b[1][3]), str(b[1][2]), str(b[0][3]), str(b[0][2])]
                coordinates = f'{r[0]},{r[1]};{r[2]},{r[3]}'
                coord_list = [float(r[1]), float(r[0]), float(r[3]), float(r[2])]
                
                res, web_link = await get_distance_osrm(coordinates, coord_list)
                
                if res['code'] == 'Ok':
                    y = res['routes'][0]
                    legs = y['legs'][0]
                    dist_meters = legs['distance']
                    dist_km = round(dist_meters / 1000, 3)

                    wsarray.cell(row=b[1][0], column=b[0][0]).value = dist_km
                    # print(b[1][0], b[1][1], b[0][0], b[0][1],  dist_km)
                else:
                    print(res['code'], coordinates, web_link)
                    wsarray.cell(row=b[1][0], column=b[0][0]).value = 0
                # next(br)
                quit += 1
    try:
        # next(br)
        pass
    except Exception:
        pass
                   
    wsarray.insert_cols(0)
    
    if flag_upload == 'csv':
        for i, b in enumerate(list(csv_data['location'])):
            wsarray.cell(row=i+1, column=1).value = b
    elif flag_upload == 'xlsx':
        for i, b in enumerate(list(dataframe['location'])):
            wsarray.cell(row=i+1, column=1).value = b

    return wsarray


if __name__ == '__main__':

    print(f'Задано ограничение по кол-ву расчетов: {calc_limit.value}')
    matrix_out = get_data_matrix()
    sheet_openpyxl = await add_data_list(matrix_out)   
    
    if flag_upload == 'csv':
        head_name = list(csv_data['location'])
    elif flag_upload == 'xlsx':
        head_name = list(dataframe['location'])
    head_name.insert(0, '*')
    new_df = pd.DataFrame(sheet_openpyxl.values, columns=head_name)
    print('ok')

Задано ограничение по кол-ву расчетов: 0
Источник данных .xlsx
ok


<h2 style="font-family:Times New Roman;color:gray">Скачать матрицу в CSV file</h2>

In [ ]:
import base64
from IPython.display import FileLink
from ipywidgets import HTML

def create_download_link(df, title = "Скачать матрицу в CSV file ", filename = "matrix_dist.csv"):  
    csv = df.to_csv(sep=';', encoding='utf-8-sig', index=False)
    b64 = base64.b64encode(csv.encode('utf-8-sig'))
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(new_df)

<h2 style="font-family:Times New Roman;">Скачать матрицу в Excel .xlsx</h2>

In [66]:
import base64
from IPython.display import FileLink
from ipywidgets import HTML
from pandas import ExcelFile, ExcelWriter


def create_download_link_excel(df, title = "Скачайте матрицу расстояний в Excel .xlsx", filename = "matrix_dist.xlsx"):  
    output = io.BytesIO()
    with ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='array', index=False)
    
    excel_data = output.getvalue()
    b64 = base64.b64encode(excel_data)
    payload = b64.decode()
    
    html = '<a download="{filename}" href="data:text/xml;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    
    return HTML(html)

create_download_link_excel(new_df)

HTML(value='<a download="matrix_dist.xlsx" href="data:text/xml;base64,UEsDBBQAAAAIAKIBJVYHQU1igQAAALEAAAAQAAAA…

<h2 style="font-family:Times New Roman;">Создать ZIP и сохранить как файл в папку temp</h2>

In [ ]:
import os
try:
    tmpath = f'temp'
    os.mkdir(tmpath)
except Exception as ex:
    pass

compression_opts = dict(method='zip', archive_name='out.csv')  
new_df.to_csv(f'{tmpath}/out.zip', index=False, compression=compression_opts) 

<h2 style="font-family:Times New Roman;">Создать (XLSX) и сохранить как файл в папку temp</h2>

In [ ]:
import os
try:
    tmpath = f'temp'
    os.mkdir(tmpath)
except Exception as ex:
    pass

file_name = 'MarksData.xlsx'
new_df.to_excel(f'{tmpath}/{file_name}')
print('DataFrame is written to Excel File successfully.')

In [59]:
import json
from pyodide import to_js
from IPython.display import JSON
from js import Object, fetch
import asyncio

async def ss():
    coordinates = '30.613808,46.471085;29.234447,47.997272'
    r1 = f'https://routing.openstreetmap.de/{type_driving}/route/v1/driving/{coordinates}?overview=false&alternatives=true&steps=false'
    resp = await fetch(r1, method="GET", headers=Object.fromEntries(to_js({ "Content-Type": "application/json" })),)
    res = await resp.text()
    res_json = json.loads(res)
    
    return res_json
    
r = await ss()
# loop = asyncio.get_event_loop()
# loop.run_until_complete(ss())
print(r)

{'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'summary': '', 'weight': 58057.9, 'duration': 58057.9, 'distance': 244351.7}], 'weight_name': 'duration', 'weight': 58057.9, 'duration': 58057.9, 'distance': 244351.7}, {'legs': [{'steps': [], 'summary': '', 'weight': 61776, 'duration': 61776, 'distance': 260646.5}], 'weight_name': 'duration', 'weight': 61776, 'duration': 61776, 'distance': 260646.5}], 'waypoints': [{'hint': 'KE1ch0FNXIcJAQAADgEAAAAAAAAAAAAAgMPdQgZ44EIAAAAAAAAAAAkBAAAOAQAAAAAAAAAAAADgDgAAqiLTAc8XxQIwIdMBrRfFAgAADxY0SkXS', 'distance': 29.274553386, 'name': 'Авангардовська вулиця', 'location': [30.614186, 46.471119]}, {'hint': 'UszXhjzNBokdAQAANAEAAAAAAAAAAAAAMx_uQqwuAEMAAAAAAAAAAB0BAAA0AQAAAAAAAAAAAADgDgAA7xG-ASVj3AIPFb4BWGHcAgAAjwI0SkXS', 'distance': 78.708342376, 'name': 'Крутівська вулиця', 'location': [29.233647, 47.997733]}]}
